In [65]:
import pandas as pd

data: str = '../../data/raw/wordle_withcritic_new_raw.jsonl'
save_path: str = '../../data/processed/wordle_withclue_new_processed.jsonl'
df = pd.read_json(data, lines=True)

# This works for wordle and wordle with clue

## Filter all conversations that are successful and have an invalid format inside the assistant text

In [66]:
# Define the text to filter out
specific_text = 'INVALID_FORMAT'

# Function to check if specific_text is in any of the dictionaries in the list
def contains_specific_text(row):
      return any(specific_text in d['content'] for d in row)

In [67]:
# Filter the DataFrame
filtered_data = df[~df['chat'].apply(contains_specific_text)]

KeyError: 'chat'

In [68]:
len(df), len(filtered_data)

(1475, 3760)

## Finding:
Wordle With clue and critic has a lot of INVALID_FORMAT in its explanations. When this is used to train the model to deliver content, 
it will learn to produce only the INVALID_CONTENT token. This does not help the game to be better played

In [62]:
filtered_data = filtered_data.rename(columns={'target_word': 'target'})
filtered_data = filtered_data.drop(['target_word_difficulty', 'target_word_clue'], axis=1)

In [63]:
filtered_data.to_json(save_path, orient='records', lines=True)

In [64]:
filtered_data.head()

,game,game_id,model,benchmark_version,experiment,episode,Aborted,Lose,Success,chat,target
0,wordle_withclue,1,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_clue_no_critic,episode_0,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",start
1,wordle_withclue,2,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_clue_no_critic,episode_1,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",share
2,wordle_withclue,3,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_clue_no_critic,episode_2,0,1,0,"[{'role': 'user', 'content': 'You are a langua...",third
3,wordle_withclue,4,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_clue_no_critic,episode_3,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",photo
4,wordle_withclue,5,CodeLlama-34b-Instruct-hf-t0.0--CodeLlama-34b-...,v1.5,0_high_frequency_words_clue_no_critic,episode_4,1,0,0,"[{'role': 'user', 'content': 'You are a langua...",might
